In [1]:
import pandas as pd
import xlrd
import xlwt
from xlutils.copy import copy
import openpyxl

In [2]:
dfpl = pd.read_excel('supermarket_sales.xls', sheet_name='ProductLine')
dfc = pd.read_excel('supermarket_sales.xls', sheet_name='city')
dfp = pd.read_excel('supermarket_sales.xls', sheet_name='product')
df = pd.read_excel('supermarket_sales.xls', sheet_name='supermarket_sales')

In [3]:
df1 = df[df['Branch'] == 'A']
df2 = df[df['Branch'] == 'B']
df3 = df[df['Branch'] == 'C']

### Source 1

- Tách và giữ Branch A

In [4]:
data = pd.read_excel('supermarket_sales.xls', sheet_name=None)
writer = pd.ExcelWriter('supermarket_sales_A.xlsx', engine='openpyxl')
for sheet_name, df in data.items():
    df.to_excel(writer, sheet_name=sheet_name, index=False)

writer.close()

In [5]:
file_path = "supermarket_sales_A.xlsx"
sheet_name = "supermarket_sales"

workbook = openpyxl.load_workbook(file_path)

sheet = workbook[sheet_name]
sheet.delete_rows(1, sheet.max_row)

data_rows = [list(row) for row in df1.values]
    
sheet.append(list(df1.columns))
for row in data_rows:
    sheet.append(row)
workbook.save(file_path)

### Source 2

- Lấy Branch B
- Merge ProductID, ProductLine
- Còn 2 sheet supermarket_sales, city

In [6]:
df2 = pd.merge(df2, dfp, on='ProductID')
df2 = df2.drop(['ProductID'], axis=1)
df2 = pd.merge(df2, dfpl, left_on='ProductLine', right_on='ProductLineID')
df2 = df2.drop(['ProductLine', 'ProductLineID'], axis=1)

In [7]:
df2

,Invoice ID,Branch,Customer type,Gender,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating,Unit price,Product line
0,529-56-3975,B,Member,M,4,5.1020,107.1420,2020-03-09,17:03:00,Cash,102.04,4.761905,5.1020,6.8,24.49,Food and beverages
1,347-34-2234,B,Member,Female,9,24.7815,520.4115,2019-02-03,13:40:00,Ewallet,495.63,4.761905,24.7815,10.0,56.69,Food and beverages
2,219-22-9386,B,Member,Male,9,44.9820,944.6220,2019-03-09,17:26:00,Credit card,899.64,4.761905,44.9820,4.2,16.49,Food and beverages
3,608-27-6295,B,Member,Male,6,15.8670,333.2070,2019-01-19,17:34:00,Credit card,317.34,4.761905,15.8670,9.8,97.22,Food and beverages
4,732-04-5373,B,Member,Male,4,9.2940,195.1740,2019-02-08,10:53:00,Cash,185.88,4.761905,9.2940,7.0,80.05,Food and beverages
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,699-88-1972,B,Normal,Male,8,39.6640,832.9440,2019-01-28,17:47:00,Credit card,793.28,4.761905,39.6640,4.2,36.51,Fashion accessories
334,509-10-0516,B,Normal,Male,4,9.1940,193.0740,2019-02-09,12:02:00,Ewallet,183.88,4.761905,9.1940,5.1,21.12,Fashion accessories
335,226-34-0034,B,Normal,Female,4,2.7560,57.8760,2019-01-10,11:10:00,Ewallet,55.12,4.761905,2.7560,9.0,28.38,Fashion accessories
336,452-04-8808,B,Normal,Male,7,30.4780,640.0380,2019-01-26,15:17:00,Cash,609.56,4.761905,30.4780,5.5,25.31,Fashion accessories


In [8]:
writer = pd.ExcelWriter('supermarket_sales_B.xlsx', engine='openpyxl')
df2.to_excel(writer, sheet_name='supermarket_sales', index=False)
dfc.to_excel(writer, sheet_name='city', index=False)
writer.close()

### Source 3

- Lấy Branch C
- Tách day, month, year
- Payment năm 2019 lưu tắt (Cash, Ewallet, Credit card) -> (C, E, CC)
- merge ProductID, City
- còn 2 sheet supermarket_sales, ProductLine

In [9]:
df3 = pd.merge(df3, dfp, on='ProductID')
df3 = df3.drop(['ProductID'], axis=1)
df3 = pd.merge(df3, dfc, on='Branch')
df3 = df3.drop(['Branch'], axis=1)

In [10]:
df3['Day'] = df3['Date'].dt.day
df3['Month'] = df3['Date'].dt.month
df3['Year'] = df3['Date'].dt.year
df3 = df3.drop(['Date'], axis=1)
df3.loc[df3['Year'] == 2019, 'Payment'] = df3.loc[df3['Year'] == 2019, 'Payment'].map({'Cash': 'C', 'Ewallet': 'E', 'Credit card': 'CC'})

In [11]:
df3

,Invoice ID,Customer type,Gender,Quantity,Tax 5%,Total,Time,Payment,cogs,gross margin percentage,gross income,Rating,Unit price,ProductLine,City,Day,Month,Year
0,300-71-4605,Member,Male,5,21.5100,451.7100,11:24:00,E,430.20,4.761905,21.5100,4.8,25.45,PD002,Naypyitaw,25,2,2019
1,183-56-6882,Member,Female,4,19.8840,417.5640,10:42:00,E,397.68,4.761905,19.8840,7.5,67.77,PD002,Naypyitaw,6,2,2019
2,232-16-2483,Member,Female,1,3.4060,71.5260,12:28:00,E,68.12,4.761905,3.4060,6.8,63.71,PD001,Naypyitaw,7,1,2019
3,554-53-8700,Member,Male,2,5.6110,117.8310,10:11:00,C,112.22,4.761905,5.6110,6.3,93.72,PD003,Naypyitaw,2,2,2019
4,228-96-1411,Member,Female,8,39.4800,829.0800,20:39:00,C,789.60,4.761905,39.4800,7.6,14.76,PD004,Naypyitaw,4,3,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,744-82-9138,Normal,Male,2,8.6130,180.8730,17:59:00,C,172.26,4.761905,8.6130,8.2,84.07,PD001,Naypyitaw,7,2,2019
325,867-47-1948,Normal,Female,10,7.9000,165.9000,12:07:00,C,158.00,4.761905,7.9000,7.8,84.07,PD001,Naypyitaw,9,1,2019
326,148-41-7930,Normal,Male,7,34.9860,734.7060,10:33:00,C,699.72,4.761905,34.9860,6.1,21.58,PD004,Naypyitaw,23,1,2019
327,189-40-5216,Normal,Male,7,33.7295,708.3195,11:40:00,C,674.59,4.761905,33.7295,6.0,21.58,PD004,Naypyitaw,9,1,2019


In [12]:
writer = pd.ExcelWriter('supermarket_sales_C.xlsx', engine='openpyxl')
df3.to_excel(writer, sheet_name='supermarket_sales', index=False)
dfpl.to_excel(writer, sheet_name='ProductLine', index=False)

writer.close()